# Initial Resource Provisioning

This interactive notebook demonstrates how to provision initial laboratory resources using the templates created in the previous step.

## Prerequisites

Before running this notebook, ensure you have completed:
- ✅ **01_service_orchestration.ipynb** - All MADSci services are running
- ✅ **02_resource_templates.ipynb** - Resource templates are created

## Overview

Resource provisioning involves:
- Creating specific instances of laboratory resources from templates
- Organizing them spatially within the lab
- Establishing proper ownership context for tracking
- Setting up resource hierarchies and relationships

## Setup and Imports

In [ ]:
# Import required modules
import sys
from datetime import datetime, timedelta
from pathlib import Path

try:
    from madsci.client.resource_client import ResourceClient
    from madsci.common.types.context_types import OwnershipInfo
    from madsci.common.types.resource_types import Resource
    from madsci.common.types.resource_types.resource_enums import ResourceStatusEnum

    print("✅ All MADSci modules imported successfully")

except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("Make sure MADSci is installed and services are running")
    sys.exit(1)

In [ ]:
# Initialize client
try:
    client = ResourceClient(resource_server_url="http://localhost:8003")

    # Test connectivity and check existing templates
    templates = client.list_templates()
    print(f"✅ Resource client initialized - {len(templates)} templates available")

    # List available templates
    print("\n📋 Available templates:")
    for template in templates:
        tags = ", ".join(
            client.get_template_info(template.resource_name).get("tags", [])
        )
        print(f"   • {template.resource_name} ({tags})")

except Exception as e:
    print(f"❌ Client initialization failed: {e}")
    print("Check that the Resource Manager service is running and templates exist")

## Resource Hierarchy Overview

MADSci supports hierarchical resource relationships:

```
Lab
├── 📍 Locations (physical spaces)
│   ├── 🔬 Instruments (liquid handlers, plate readers, etc.)
│   └── 🏠 Storage Areas (incubators, refrigerators, shelves)
├── 📦 Containers (plates, tip racks, tubes)
│   ├── 🕳️ Slots/Wells (individual wells in plates)
│   └── 📚 Stacks (multiple plates in a stack)
└── 🧪 Consumables (reagents, tips, samples)
```

Let's start by setting up the physical locations:

## 1. Location Setup

First, let's establish physical locations within the lab:

In [ ]:
# Define ownership context for resource provisioning
setup_owner = OwnershipInfo(
    node="resource_provisioner",
    experiment="initial_lab_setup",
    user="lab_administrator",
)

print("🏷️ Provisioning context:")
print(f"   Node: {setup_owner.node}")
print(f"   Experiment: {setup_owner.experiment}")
print(f"   User: {setup_owner.user}")

In [ ]:
# Create instrument location resources
locations_to_create = [
    {
        "name": "LiquidHandler_1_Deck",
        "class": "InstrumentDeck",
        "attributes": {
            "instrument_type": "liquid_handler",
            "deck_positions": 8,
            "max_stack_height": 5,
            "temperature_controlled": False,
            "compatible_labware": ["96-well-plate", "384-well-plate", "tip-rack"],
        },
    },
    {
        "name": "PlateReader_1_Hotel",
        "class": "PlateHotel",
        "attributes": {
            "instrument_type": "plate_reader",
            "hotel_slots": 50,
            "temperature_controlled": True,
            "temperature_range": [4, 45],
            "compatible_plates": ["96-well", "384-well"],
        },
    },
    {
        "name": "Incubator_37C_001",
        "class": "EnvironmentalChamber",
        "attributes": {
            "chamber_type": "incubator",
            "capacity": 20,  # plates
            "temperature_setpoint": 37,
            "co2_control": True,
            "humidity_control": True,
        },
    },
    {
        "name": "Refrigerator_4C_001",
        "class": "StorageUnit",
        "attributes": {
            "storage_type": "refrigerator",
            "temperature_setpoint": 4,
            "capacity": 100,  # containers
            "shelves": 4,
        },
    },
]

created_locations = []

print("🏗️ Creating laboratory locations...")
print("=" * 40)

for location_config in locations_to_create:
    try:
        location = Resource(
            resource_name=location_config["name"],
            resource_class=location_config["class"],
            attributes=location_config["attributes"],
            owner=setup_owner,
            status=ResourceStatusEnum.available,
        )

        created_location = client.add_resource(location)
        created_locations.append(created_location)

        # Display key attributes
        key_attrs = {
            k: v
            for k, v in location.attributes.items()
            if k
            in ["instrument_type", "capacity", "temperature_setpoint", "deck_positions"]
        }
        attr_str = ", ".join([f"{k}: {v}" for k, v in key_attrs.items()])

        print(f"✅ Created {location.resource_name}")
        print(f"   ID: {created_location.resource_id[:8]}... ({attr_str})")

    except Exception as e:
        print(f"❌ Failed to create {location_config['name']}: {e}")

print(f"\n📊 Successfully created {len(created_locations)} locations")

## 2. Assay Plate Provisioning

Now let's create plates for different types of experiments:

In [ ]:
# Create assay plates for high-throughput screening
plate_batches = [
    {
        "template": "standard_96well_plate",
        "prefix": "AssayPlate",
        "count": 15,
        "batch_number": "AP20241201",
        "plate_type": "assay",
        "owner": OwnershipInfo(
            node="liquid_handler_1",
            experiment="screening_campaign_001",
            user="researcher_smith",
        ),
    },
    {
        "template": "standard_96well_plate",
        "prefix": "ControlPlate",
        "count": 5,
        "batch_number": "CP20241201",
        "plate_type": "control",
        "owner": OwnershipInfo(
            node="quality_control", experiment="qc_validation", user="qc_technician"
        ),
    },
]

all_created_plates = []

print("🧪 Provisioning laboratory plates...")
print("=" * 40)

for batch in plate_batches:
    print(f"\n📦 Creating {batch['count']} {batch['prefix']} plates...")
    batch_plates = []

    for i in range(1, batch["count"] + 1):
        try:
            plate = client.create_resource_from_template(
                template_name=batch["template"],
                resource_name=f"{batch['prefix']}_{i:03d}",
                overrides={
                    "attributes": {
                        "batch_number": batch["batch_number"],
                        "plate_type": batch["plate_type"],
                        "coating": "tissue_culture",
                        "sterile": True,
                        "prepared_date": datetime.now().isoformat()[:10],
                        "usage_count": 0,
                        "last_wash": None,
                        "quality_checked": True,
                    },
                    "owner": batch["owner"].model_dump(),
                    "status": ResourceStatusEnum.available,
                },
                add_to_database=True,
            )

            batch_plates.append(plate)
            all_created_plates.append(plate)

            if i % 5 == 0:  # Progress indicator
                print(f"   ✓ Created {i} plates...")

        except Exception as e:
            print(f"   ❌ Failed to create {batch['prefix']}_{i:03d}: {e}")

    success_rate = len(batch_plates) / batch["count"] * 100
    print(
        f"   📊 {batch['prefix']} batch: {len(batch_plates)}/{batch['count']} plates ({success_rate:.0f}% success)"
    )

print(f"\n🎉 Total plates created: {len(all_created_plates)}")

## 3. Consumable Provisioning

Let's create tip racks and reagents:

In [ ]:
# Create tip racks for liquid handling
tip_rack_owner = OwnershipInfo(
    node="liquid_handler_1", experiment="consumable_management", user="lab_technician"
)

created_tip_racks = []

print("💧 Provisioning tip racks...")
print("=" * 30)

# Check if tip rack template exists
tip_templates = [t for t in templates if "tip" in t.resource_name.lower()]
if tip_templates:
    tip_template_name = tip_templates[0].resource_name
    print(f"Using template: {tip_template_name}")

    for i in range(1, 11):  # Create 10 tip racks
        try:
            tip_rack = client.create_resource_from_template(
                template_name=tip_template_name,
                resource_name=f"TipRack_200uL_{i:02d}",
                overrides={
                    "attributes": {
                        "lot_number": "TR200-20241201",
                        "tip_count": 96,
                        "sterile": True,
                        "opened": False,
                        "remaining_tips": 96,
                    },
                    "owner": tip_rack_owner.model_dump(),
                    "status": ResourceStatusEnum.available,
                },
                add_to_database=True,
            )

            created_tip_racks.append(tip_rack)

        except Exception as e:
            print(f"❌ Failed to create TipRack_200uL_{i:02d}: {e}")

    print(f"✅ Created {len(created_tip_racks)} tip racks")
else:
    print("⚠️  No tip rack template found - skipping tip rack provisioning")
    created_tip_racks = []

In [ ]:
# Create reagent inventory
reagent_owner = OwnershipInfo(
    node="reagent_manager", experiment="reagent_inventory", user="chemistry_team"
)

reagent_recipes = [
    {
        "name": "PBS_1X_Buffer_001",
        "type": "buffer",
        "concentration": "1X",
        "volume": 1000.0,  # mL
        "storage_temp": 4,
        "expiry_months": 12,
    },
    {
        "name": "DMSO_Stock_001",
        "type": "solvent",
        "concentration": "100%",
        "volume": 500.0,
        "storage_temp": 25,
        "expiry_months": 24,
    },
    {
        "name": "AssayBuffer_2X_001",
        "type": "assay_buffer",
        "concentration": "2X",
        "volume": 100.0,
        "storage_temp": -20,
        "expiry_months": 6,
    },
    {
        "name": "Trypsin_EDTA_001",
        "type": "enzyme",
        "concentration": "0.25%",
        "volume": 100.0,
        "storage_temp": -20,
        "expiry_months": 12,
    },
    {
        "name": "Media_DMEM_001",
        "type": "cell_culture_media",
        "concentration": "1X",
        "volume": 500.0,
        "storage_temp": 4,
        "expiry_months": 3,
    },
]

created_reagents = []

print("🧪 Provisioning reagent inventory...")
print("=" * 40)

# Check if reagent template exists
reagent_templates = [t for t in templates if "reagent" in t.resource_name.lower()]
if reagent_templates:
    reagent_template_name = reagent_templates[0].resource_name
    print(f"Using template: {reagent_template_name}\n")

    for recipe in reagent_recipes:
        try:
            expiry_date = (
                datetime.now() + timedelta(days=30 * recipe["expiry_months"])
            ).isoformat()[:10]

            reagent = client.create_resource_from_template(
                template_name=reagent_template_name,
                resource_name=recipe["name"],
                overrides={
                    "attributes": {
                        "reagent_type": recipe["type"],
                        "concentration": recipe["concentration"],
                        "volume": recipe["volume"],
                        "remaining_volume": recipe["volume"],
                        "storage_temp": recipe["storage_temp"],
                        "lot_number": f"LOT{datetime.now().strftime('%Y%m%d')}",
                        "expiry_date": expiry_date,
                        "opened": False,
                        "received_date": datetime.now().isoformat()[:10],
                    },
                    "owner": reagent_owner.model_dump(),
                    "status": ResourceStatusEnum.available,
                },
                add_to_database=True,
            )

            created_reagents.append(reagent)
            print(
                f"✅ {recipe['name']} ({recipe['volume']}mL, {recipe['concentration']}, {recipe['storage_temp']}°C)"
            )

        except Exception as e:
            print(f"❌ Failed to create {recipe['name']}: {e}")

    print(f"\n📊 Created {len(created_reagents)} reagents")
else:
    print("⚠️  No reagent template found - skipping reagent provisioning")
    created_reagents = []

## 4. Resource Spatial Organization

Now let's organize resources in physical locations:

In [ ]:
# Assign plates to specific locations
print("🗂️ Organizing resources in physical locations...")
print("=" * 50)

# Find our created locations
liquid_handler_deck = next(
    (loc for loc in created_locations if "LiquidHandler" in loc.resource_name), None
)
plate_reader_hotel = next(
    (loc for loc in created_locations if "PlateReader" in loc.resource_name), None
)
incubator = next(
    (loc for loc in created_locations if "Incubator" in loc.resource_name), None
)
refrigerator = next(
    (loc for loc in created_locations if "Refrigerator" in loc.resource_name), None
)

assigned_resources = 0

if liquid_handler_deck and len(all_created_plates) > 0:
    # Assign first 8 assay plates to liquid handler deck
    deck_plates = all_created_plates[:8]

    print(f"📍 Assigning {len(deck_plates)} plates to liquid handler deck:")
    for i, plate in enumerate(deck_plates):
        position = i + 1
        try:
            updated_plate = client.update_resource(
                plate.resource_id,
                {
                    "attributes": {
                        **plate.attributes,
                        "location_id": liquid_handler_deck.resource_id,
                        "deck_position": position,
                        "coordinates": {"x": position % 4, "y": position // 4},
                    }
                },
            )
            print(f"   ✓ {plate.resource_name} → Position {position}")
            assigned_resources += 1
        except Exception as e:
            print(f"   ❌ Failed to assign {plate.resource_name}: {e}")

if plate_reader_hotel and len(all_created_plates) > 8:
    # Assign next 10 plates to plate reader hotel
    hotel_plates = all_created_plates[8:18]

    print(f"\n🏨 Storing {len(hotel_plates)} plates in plate reader hotel:")
    for i, plate in enumerate(hotel_plates):
        slot = i + 1
        try:
            updated_plate = client.update_resource(
                plate.resource_id,
                {
                    "attributes": {
                        **plate.attributes,
                        "location_id": plate_reader_hotel.resource_id,
                        "hotel_slot": slot,
                        "stored_at": datetime.now().isoformat(),
                    }
                },
            )
            if i < 3 or i >= len(hotel_plates) - 2:  # Show first 3 and last 2
                print(f"   ✓ {plate.resource_name} → Slot {slot}")
            elif i == 3:
                print(f"   ... (storing {len(hotel_plates) - 5} more plates)")
            assigned_resources += 1
        except Exception as e:
            print(f"   ❌ Failed to store {plate.resource_name}: {e}")

# Assign reagents to appropriate storage
if refrigerator and created_reagents:
    cold_reagents = [
        r for r in created_reagents if r.attributes.get("storage_temp", 25) <= 4
    ]

    if cold_reagents:
        print(f"\n🧊 Storing {len(cold_reagents)} cold reagents in refrigerator:")
        for i, reagent in enumerate(cold_reagents):
            shelf = (i // 5) + 1  # 5 reagents per shelf
            try:
                client.update_resource(
                    reagent.resource_id,
                    {
                        "attributes": {
                            **reagent.attributes,
                            "location_id": refrigerator.resource_id,
                            "shelf_position": shelf,
                            "stored_at": datetime.now().isoformat(),
                        }
                    },
                )
                temp = reagent.attributes.get("storage_temp", "Unknown")
                print(f"   ✓ {reagent.resource_name} → Shelf {shelf} ({temp}°C)")
                assigned_resources += 1
            except Exception as e:
                print(f"   ❌ Failed to store {reagent.resource_name}: {e}")

print(f"\n📊 Total resources assigned to locations: {assigned_resources}")

## 5. Resource Stacking and Grouping

Let's create plate stacks for efficient storage:

In [ ]:
# Create plate stacks for remaining plates
remaining_plates = all_created_plates[18:]  # Plates not assigned to instruments

if len(remaining_plates) >= 3:
    print(f"📚 Creating plate stacks for {len(remaining_plates)} remaining plates...")
    print("=" * 55)

    # Group plates into stacks of 5
    stack_size = 5
    stacks_created = []

    for stack_num in range(0, len(remaining_plates), stack_size):
        stack_plates = remaining_plates[stack_num : stack_num + stack_size]

        if len(stack_plates) < 2:  # Don't create single-plate stacks
            break

        try:
            # Create stack resource
            stack = Resource(
                resource_name=f"PlateStack_{stack_num // stack_size + 1:02d}",
                resource_class="PlateStack",
                attributes={
                    "stack_height": len(stack_plates),
                    "max_height": 10,
                    "plate_type": "96-well",
                    "stack_contents": [p.resource_name for p in stack_plates],
                },
                owner=setup_owner,
                status=ResourceStatusEnum.available,
            )

            created_stack = client.add_resource(stack)
            stacks_created.append(created_stack)

            print(
                f"✅ Created {created_stack.resource_name} with {len(stack_plates)} plates"
            )

            # Assign plates to stack
            for level, plate in enumerate(stack_plates, 1):
                try:
                    client.update_resource(
                        plate.resource_id,
                        {
                            "attributes": {
                                **plate.attributes,
                                "parent_stack_id": created_stack.resource_id,
                                "stack_level": level,
                                "stacked_at": datetime.now().isoformat(),
                            }
                        },
                    )
                    print(f"   • {plate.resource_name} → Level {level}")
                except Exception as e:
                    print(f"   ❌ Failed to stack {plate.resource_name}: {e}")

            print()  # Empty line between stacks

        except Exception as e:
            print(f"❌ Failed to create stack: {e}")

    print(f"📊 Created {len(stacks_created)} plate stacks")
else:
    print("ℹ️ Not enough remaining plates to create stacks (minimum 3 required)")
    stacks_created = []

## 6. Resource Inventory Summary

Let's generate a comprehensive inventory report:

In [ ]:
# Generate comprehensive inventory report
from typing import List


def generate_inventory_report() -> List[Resource]:
    """Generate a detailed inventory report."""
    try:
        # Get all resources
        all_resources = client.list_resources()

        # Filter resources created in this session
        session_experiments = [
            "initial_lab_setup",
            "screening_campaign_001",
            "qc_validation",
            "reagent_inventory",
            "consumable_management",
        ]

        session_resources = []
        for resource in all_resources:
            if resource.owner and resource.owner.experiment in session_experiments:
                session_resources.append(resource)

        return session_resources

    except Exception as e:
        print(f"❌ Failed to generate inventory: {e}")
        return []


inventory = generate_inventory_report()

print("📦 Laboratory Inventory Report")
print("=" * 50)
print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total resources: {len(inventory)}")
print()

if inventory:
    # Categorize resources
    categories = {
        "Locations": [],
        "Plates": [],
        "Reagents": [],
        "Tips": [],
        "Stacks": [],
        "Other": [],
    }

    for resource in inventory:
        name = resource.resource_name
        if any(
            loc_type in name
            for loc_type in ["Deck", "Hotel", "Incubator", "Refrigerator"]
        ):
            categories["Locations"].append(resource)
        elif "Plate" in name and "Stack" not in name:
            categories["Plates"].append(resource)
        elif any(
            reagent in name for reagent in ["PBS", "DMSO", "Buffer", "Trypsin", "Media"]
        ):
            categories["Reagents"].append(resource)
        elif "Tip" in name:
            categories["Tips"].append(resource)
        elif "Stack" in name:
            categories["Stacks"].append(resource)
        else:
            categories["Other"].append(resource)

    # Display categorized inventory
    for category, resources in categories.items():
        if resources:
            print(f"🔸 {category} ({len(resources)}):")

            for resource in resources[:5]:  # Show first 5 in each category
                status = resource.status if hasattr(resource, "status") else "unknown"
                owner = resource.owner.node if resource.owner else "unassigned"

                # Show key attributes based on category
                extra_info = ""
                if resource.attributes:
                    if category == "Plates":
                        batch = resource.attributes.get("batch_number", "")
                        plate_type = resource.attributes.get("plate_type", "")
                        extra_info = f" (Batch: {batch}, Type: {plate_type})"
                    elif category == "Reagents":
                        volume = resource.attributes.get("volume", "")
                        temp = resource.attributes.get("storage_temp", "")
                        extra_info = f" ({volume}mL, {temp}°C)"
                    elif category == "Locations":
                        capacity = resource.attributes.get(
                            "capacity"
                        ) or resource.attributes.get("deck_positions")
                        if capacity:
                            extra_info = f" (Capacity: {capacity})"

                print(f"   • {resource.resource_name} [{status}] → {owner}{extra_info}")

            if len(resources) > 5:
                print(f"   ... and {len(resources) - 5} more")
            print()

    # Summary statistics
    print("📊 Summary Statistics:")
    for category, resources in categories.items():
        if resources:
            print(f"   {category}: {len(resources)}")

    # Resource utilization
    available = len(
        [
            r
            for r in inventory
            if hasattr(r, "status") and r.status == ResourceStatusEnum.available
        ]
    )
    total_with_status = len([r for r in inventory if hasattr(r, "status")])
    if total_with_status > 0:
        utilization = (total_with_status - available) / total_with_status * 100
        print(
            f"\n💡 Resource Utilization: {utilization:.1f}% ({total_with_status - available}/{total_with_status} in use)"
        )

else:
    print("No resources found from this provisioning session.")

## 7. Resource Validation

Let's validate that our resources are properly configured:

In [ ]:
# Resource validation checks
print("🔍 Resource Validation Checks")
print("=" * 35)

validation_results = {
    "ownership_context": 0,
    "required_attributes": 0,
    "location_assignments": 0,
    "status_values": 0,
    "template_compliance": 0,
}

issues_found = []

# Check ownership context
print("\n👥 Checking ownership context...")
for resource in inventory:
    if resource.owner and all(
        [resource.owner.node, resource.owner.experiment, resource.owner.user]
    ):
        validation_results["ownership_context"] += 1
    else:
        issues_found.append(f"Missing ownership context: {resource.resource_name}")

print(
    f"   ✅ {validation_results['ownership_context']}/{len(inventory)} resources have complete ownership context"
)

# Check required attributes
print("\n⚙️  Checking required attributes...")
for resource in inventory:
    if resource.attributes and isinstance(resource.attributes, dict):
        validation_results["required_attributes"] += 1
    else:
        issues_found.append(f"Missing attributes: {resource.resource_name}")

print(
    f"   ✅ {validation_results['required_attributes']}/{len(inventory)} resources have attributes"
)

# Check location assignments for plates
print("\n📍 Checking location assignments...")
plates_with_locations = 0
total_plates = len(
    [
        r
        for r in inventory
        if "Plate" in r.resource_name and "Stack" not in r.resource_name
    ]
)

for resource in inventory:
    if "Plate" in resource.resource_name and "Stack" not in resource.resource_name:
        if resource.attributes and any(
            loc_key in resource.attributes
            for loc_key in [
                "location_id",
                "deck_position",
                "hotel_slot",
                "parent_stack_id",
            ]
        ):
            plates_with_locations += 1
        else:
            issues_found.append(
                f"Plate not assigned to location: {resource.resource_name}"
            )

validation_results["location_assignments"] = plates_with_locations
print(f"   ✅ {plates_with_locations}/{total_plates} plates assigned to locations")

# Check status values
print("\n🔄 Checking resource status...")
for resource in inventory:
    if hasattr(resource, "status") and resource.status:
        validation_results["status_values"] += 1
    else:
        issues_found.append(f"Missing status: {resource.resource_name}")

print(
    f"   ✅ {validation_results['status_values']}/{len(inventory)} resources have status values"
)

# Summary
print("\n📋 Validation Summary:")
print("-" * 25)
total_checks = sum(validation_results.values())
max_possible = len(inventory) * len(validation_results)
if total_plates > 0:
    max_possible = (
        max_possible - len(inventory) + total_plates
    )  # Adjust for location check

success_rate = (total_checks / max_possible * 100) if max_possible > 0 else 0

print(f"Overall validation score: {success_rate:.1f}%")
print(f"Issues found: {len(issues_found)}")

if issues_found and len(issues_found) <= 5:
    print("\n⚠️  Issues detected:")
    for issue in issues_found:
        print(f"   • {issue}")
elif len(issues_found) > 5:
    print(f"\n⚠️  {len(issues_found)} issues detected (showing first 5):")
    for issue in issues_found[:5]:
        print(f"   • {issue}")

if success_rate >= 90:
    print("\n🎉 Resource provisioning validation passed!")
elif success_rate >= 75:
    print(
        "\n⚠️  Resource provisioning mostly successful, but some issues need attention."
    )
else:
    print("\n❌ Significant issues found in resource provisioning. Please review.")

## 8. Export Resource Inventory

Let's export our resource inventory for record keeping:

In [ ]:
# Export resource inventory
import json
from typing import Optional


def export_resource_inventory() -> Optional[Path]:
    """Export resource inventory to JSON file."""
    try:
        inventory_data = {
            "export_metadata": {
                "export_date": datetime.now().isoformat(),
                "exported_by": "setup_notebook",
                "lab_id": "01JVDFED2K18FVF0E7JM7SX09F",
                "provisioning_session": "initial_lab_setup",
            },
            "summary": {
                "total_resources": len(inventory),
                "locations": len(created_locations),
                "plates": len(all_created_plates),
                "tip_racks": len(created_tip_racks),
                "reagents": len(created_reagents),
            },
            "resources": [],
        }

        # Add resource details
        for resource in inventory:
            resource_data = {
                "id": resource.resource_id,
                "name": resource.resource_name,
                "class": resource.resource_class,
                "status": str(resource.status) if hasattr(resource, "status") else None,
                "attributes": resource.attributes,
                "owner": resource.owner.model_dump() if resource.owner else None,
                "created_at": datetime.now().isoformat(),  # Approximation
            }

            # Add container-specific fields
            if hasattr(resource, "capacity"):
                resource_data.update(
                    {
                        "capacity": resource.capacity,
                        "rows": getattr(resource, "rows", None),
                        "columns": getattr(resource, "columns", None),
                    }
                )

            inventory_data["resources"].append(resource_data)

        # Save to file
        export_file = Path("../requirements/initial_inventory.json")
        export_file.parent.mkdir(exist_ok=True)

        export_file.write_text(json.dumps(inventory_data, indent=2, default=str))

        return export_file

    except Exception as e:
        print(f"❌ Failed to export inventory: {e}")
        return None


exported_inventory = export_resource_inventory()

if exported_inventory:
    file_size = exported_inventory.stat().st_size / 1024
    print(f"✅ Resource inventory exported to: {exported_inventory}")
    print(f"   File size: {file_size:.1f} KB")
    print(f"   Contains {len(inventory)} resources")
else:
    print("❌ Failed to export resource inventory")

## Summary and Next Steps

🎉 **Congratulations!** You have successfully provisioned your laboratory with:

### ✅ Created Resources:
- **Locations**: Laboratory spaces and storage areas
- **Plates**: Assay and control plates with proper ownership
- **Consumables**: Tip racks and reagent inventory
- **Organization**: Spatial assignments and plate stacks

### ✅ Established Infrastructure:
- Physical location hierarchy
- Resource spatial organization
- Proper ownership context throughout
- Validation and quality checks

### ✅ Created Documentation:
- Complete resource inventory export
- Validation reports and issue tracking
- Resource utilization metrics

### Next Steps

1. **✅ Service Orchestration** - Complete
2. **✅ Resource Templates** - Complete
3. **✅ Initial Resources** - Complete (this notebook)
4. **➡️ Validation** - `04_validation.ipynb`

### Resource Management Best Practices Learned

- **Batch Operations**: Efficient creation of multiple similar resources
- **Consistent Naming**: Structured naming with counters and identifiers
- **Context Management**: Proper ownership tracking throughout
- **Spatial Organization**: Physical location assignments for workflow efficiency
- **State Management**: Appropriate status and attribute initialization
- **Validation**: Comprehensive checks for data integrity
- **Documentation**: Export capabilities for audit trails

---

**💡 Pro Tips for Resource Management:**
- Use batch operations for creating similar resources
- Maintain consistent naming conventions across experiments
- Always set proper ownership context for tracking
- Assign physical locations to enable spatial workflows
- Regular validation prevents data integrity issues
- Export inventories regularly for backup and compliance
- Monitor resource utilization to optimize lab efficiency

**🔧 Resource Lifecycle Management:**
- Resources can be updated as they're used in experiments
- Status tracking enables workflow automation
- Location changes can be tracked for audit trails
- Hierarchical relationships support complex lab setups